# Interactive NDVI

In [81]:
import datetime
import geemap
import ee
from astral import LocationInfo
from astral.sun import sun

# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'

bands = [BLUE, RED, VEGGIE]

goes_collection = ee.ImageCollection('NOAA/GOES/16/MCMIPC') 

def set_coordinates(left, bottom, right, top):
    return ee.Geometry.BBox(left, bottom, right, top)


def applyScaleandOffset(image):
    
    band_list = image.select('CMI_C..').bandNames()

    scales = (band_list.map(lambda name: ee.String(name).cat('_scale'))).map(lambda band: image.getNumber(band))
    offsets = (band_list.map(lambda name: ee.String(name).cat('_offset'))).map(lambda band: image.getNumber(band))

    scaled = image.select('CMI_C..').multiply(ee.Image.constant(scales)).add(ee.Image.constant(offsets))

    return image.addBands(srcImg = scaled, overwrite = True)


def get_daytime(lat, lon):

    loc = LocationInfo(latitude=lat, longitude=lon)
    s = sun(loc.observer, date=datetime.date.today() - datetime.timedelta(days=1))
    
    return (s['sunrise'].strftime(format="%Y-%m-%dT%H:%M:00"), s['sunset'].strftime(format="%Y-%m-%dT%H:%M:00"))


In [77]:
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(["CMI_C03", "CMI_C02"]).rename('NDVI')
    return image.addBands(ndvi)

def calculate_ndwi(image):

    nir = image.select("CMI_C03")
    red = image.select("CMI_C02")
    blue = image.select("CMI_C01")

    green = red.multiply(0.45).add(nir.multiply(0.1)).add(blue.multiply(0.45))
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')

    return image.addBands(ndwi)

In [78]:
# Set the location of the ROI. USER INPUT REQUIRED.
roi = set_coordinates(-102.84, 37.25, -96.91, 42.27)

# Get the image collection for the given time period and location.
date_start, date_end = get_daytime(42.27, -102.84) #UPPER LEFT CORNER (smallest day duration)
period = ee.DateRange(date_start, date_end)

In [83]:
# Calculate the indexes
indexes = goes_collection.filterDate(period).select(bands).map(applyScaleandOffset).map(calculate_ndvi).map(calculate_ndwi).reduce(ee.Reducer.median()).clip(roi)
#indexes.getInfo()

In [86]:

m = geemap.Map()
m.setCenter(-102, 37, 4)

# Add the image to the map.
ndviParams = {'bands': ['NDVI_median'],
                'min': -1,
                'max': 1,
                'palette': ['blue', 'white', 'green']}


ndwiParams = {'bands': ['NDWI_median'],
                'min': 0.0,
                'max': 1,
                'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']}

m.addLayer(indexes, ndviParams, 'NDVI')
m.addLayer(indexes, ndwiParams, 'NDWI')
m.addLayerControl()
m

Map(center=[37, -102], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…